In [80]:
# 必要ライブラリのインストール
from PIL import Image
import numpy as np

In [81]:
def MSE(x, y):
    l = len(x)
    sum = 0.0
    for i in range(l):
        diff = float(x[i]) - float(y[i])
        sum += (diff * diff)
    return sum / l

In [82]:
img1 = Image.open('ref/c1b_ref.tga')
img2 = Image.open('c1b_approx2/c1b_16_16_0.5_16_2.tga')
img1_gray = img1.convert('L')
img2_gray = img2.convert('L')
img1_gray_arr = np.array(img1_gray)
img2_gray_arr = np.array(img2_gray)

In [100]:
def phi(k, N):
	if k == 0:
		return np.ones(N) / np.sqrt(N)
	else:
		return np.sqrt(2.0 / N) * np.cos((k * np.pi / (2 * N)) * (np.arange(N) * 2 + 1))

def DCT_mat_reducedto1d(N):
	phi_1d = np.array([phi(k, N) for k in range(N)])
	phi_2d = np.zeros((N,N,N,N))
	
	for i in range(N):
		for j in range(N):
			phi_i, phi_j = np.meshgrid(phi_1d[i], phi_1d[j])
			phi_2d[i, j] = phi_i*phi_j
			
	return phi_2d.reshape(N * N, N * N)

def DCT_mat_jpg():
	DCT_mat = DCT_mat_element_product(8)
	res = []
	for i in range(8):
		for j in range(8):
			cond = (i == 0) or (j == 0) or (i + j <= 4)
			if cond: res.append(DCT_mat[i * 8 + j])
	return np.array(res)

In [101]:
def DCT_mat_jpg2():
	res = DCT_mat_reducedto1d(8)
	for i in range(8):
		for j in range(8):
			cond = (i == 0) or (j == 0) or (i + j <= 4)
			if not cond:
				for k in range(64): res[i * 8 + j, k] = 0.0
	return res

In [84]:
C = DCT_mat_jpg()
D = C * C

def var_ac_coef(sample_ratio):
    return np.sum(D.dot(1.0 / sample_ratio))

In [118]:
C2 = DCT_mat_reducedto1d(8).T.dot(DCT_mat_jpg2())
D2 = C2 * C2

def var_ac_coef2(sample_ratio):
    return np.sum(D2.dot(1.0 / sample_ratio))

In [123]:
import random

alpha = 0.5 # ratio of praparation
N = 10 ** 6 # number of iteration
eps = 10 ** (-6)
arr1 = np.array([alpha / 64.0] * 64)
arr2 = np.array([(1.0 - alpha) / 64.0] * 64)

for itr in range(1, N):
    i = random.randrange(64)
    j = random.randrange(63)
    if j >= i: j += 1

    sum = arr2[i] + arr2[j]
    left = 0
    right = sum

    while right - left >= eps:
        d = right - left
        c1 = left + d / 3.0
        c2 = right - d / 3.0

        arr2[i] = c1
        arr2[j] = sum - c1
        val1 = var_ac_coef2(arr1 + arr2)

        arr2[i] = c2
        arr2[j] = sum - c2
        val2 = var_ac_coef2(arr1 + arr2)

        if val1 < val2: right = c2
        else: left = c1
    
    arr2[i] = (left + right) / 2.0
    arr2[j] = sum - arr2[i]

    if itr % 10000 == 0: print(itr)

arr3 = arr1 + arr2
print(arr3)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
[0.01981296 0.01646633 0.01636557 0.01633258 0.01633282 0.01636567
 0.01646632 0.01981256 0.01646629 0.01452945 0.01501102 0.0148598
 0.01486014 0.01501069 0.01452951 0.01646618 0.01636553 0.01501093
 0.01460399 0.01417557 0.01417535 0.01460417 0.01501081 0.01636576
 0.01633262 0.01485987 0.01417529 0.01463227 0.01

In [125]:
arr10 = np.array([alpha / 64.0] * 64)
arr20 = np.array([(1.0 - alpha) / 64.0] * 64)
arr30 = arr10 + arr20
print('normal: ' + str(var_ac_coef2(arr30) / 64.0))
print('opt:    ' + str(var_ac_coef2(arr3) / 64.0))

normal: 20.99999999999999
opt:    20.83779711687421


In [105]:
def dHondt_method(n_votes, r):
    # n_votes:票数　r:当選数
    h = np.zeros_like(n_votes)  # 配分（最初はすべて0）
    for _ in range(r):
        a = n_votes / (h + 1)
        h[np.argmax(a)] += 1
    return h.astype('int')

In [127]:
arr4 = dHondt_method(arr3, 100000 * 64)
arr5 = arr4.reshape(8, 8)
for i in range(8):
    for j in range(8):
        print(arr5[i, j], end = '\t')
    print('')

126803	105385	104740	104529	104530	104740	105384	126801	
105384	92988	96070	95103	95105	96068	92989	105384	
104739	96070	93465	90724	90722	93467	96069	104741	
104529	95103	90722	93646	93646	90722	95104	104528	
104528	95104	90722	93646	93646	90723	95103	104529	
104740	96069	93468	90722	90722	93466	96072	104740	
105384	92988	96071	95104	95103	96071	92987	105385	
126801	105384	104738	104529	104530	104741	105384	126800	


In [69]:
np.array([12, -8, 5])

array([12, -8,  5])